# 1. 컨텐츠 기반 모델

정의 : 사용자가 이전에 구매한 상품중에서 좋아하는 상품들과 `유사한 상품`을 추천하는 방법

<br>

- Represented Items<br>
아이템을 벡터 형태로 표현

# 2. 유사도 함수

## 2-1. 유클리디안 유사도

문서간의 유사도 계산

<br>

$ ||p - q|| =  \sqrt{(p - q)(p - q)} = \sqrt{||p||^2 + ||q||^2 - 2 p q}$<br>

$ 유클리디안 \, 유사도 = \frac{1}{유클리디안 \, 거리 + 1e ^ -05}$

<br>

- 장점 : 계산하기 쉬움
- 단점 : p와 q의 분포가 다르거나 범위가 다른 경우에 상관성 놓침


## 2-2 코사인 유사도

문서간의 유사도 계산 

<br>

$ cos(\theta) = \frac{A \, B}{||A|| \, ||B||} = \frac{\sum_{i=1}^{n} A_iB_i}{\sqrt{\sum_{i=1}^{n} A_i ^2} \, \sqrt{\sum_{i=1}^{n} B_i ^2}}$

<br>

- 장점 : 벡터의 크기가 중요하지 않은 경우에 거리를 측정하기 위한 metric으로 사용<br>
ex. 문서내에서 단어의 빈도수와 같이 문서들의 길이가 고르지 않아도 문서내에서 얼마나 나왔는지와 같은 비율을 확인하기 때문에 상관 없음
- 단점 : 벡터의 크기가 중요한 경우에 대해서 잘 작동하지 않음

## 2-3. 피어슨 유사도

상관관계를 분석하기 위한 유사도 

<br>

$ rXY = \frac{\sum_{1}^{n}(X_i - \bar X)(Y_i - \bar Y)}{\sqrt{\sum_{1}^{n} (X_i - \bar X) ^2} \sqrt{\sum_{1}^{n} (Y_i - \bar Y) ^2}}$

## 2-4. 자카드 유사도

집합에서 얼마만큼 결합된 부분이 있는지 확인하는 유사도

<br>

$J(A,B) = \frac{|A \cap B|}{|A \cup B|} = \frac{|A \cap B|}{|A| + |B| - |A \cap B|}$

# 3. 벡터화

## 3-1. TF-IDF

**정의**<br>
특정 문서 내에 특정 단어가 얼마나 자주 등장하는지를 의미하는 단어 빈도(TF)와 <br>
전체 문서에서 특정 단어가 얼마나 자주 등장하는지를 의미하는 역문서 빈도(DF)를 통해 <br>
`다른 문서에서는 등장하지 않지만 특정 문서에서만 자주 등장하는 단어`를 찾아 <br>
문서 내 단어의 가중치를 계산하는 방법 

<br>

문서의 핵심어 추출, 문서들 사이의 유사도 계산, 검색 결과의 중요도 정하는 작업 등에 활용

- TF(d, t) : 특정 문서 d에서의 특정 단어 t의 등장 횟수
- DF(t) : 특정 단어 t가 등장한 문서의 수 
- IDF(d, t) : $IDF(d, t) = log{\frac{n}{1 + DF(t)}}$<br>
$ TF(d, t) * IDF(d, t) = TF-IDF(d, t) $

**TF-IDF를 사용하는 이유**
<br>

빈도수를 기반으로 많이 나오는 중요한 단어를 잡아주는 방법을 Count Vectorizer라고 함<br>
하지만 Count Vectorizer는 단순 빈도만을 계산하기 때문에 조사나 관사처럼 의미는 없지만 문장에 많이 등장하는 단어들도 가중치를 두는 한계가 있음<br>
이때 TF-IDF를 사용하면 조사나 관사에 패널티를 줘서 적절하게 중요한 단어만을 잡아냄

- 장점 : 직관적인 해석이 가능함
- 단점 : 대규모 말뭉치를 다룰 때 메모리상의 문제가 발생
     - 높은 차원을 가짐
     - 매우 sparse한 형태의 데이터

### 3-1-1. 벡터화 실습

In [1]:
docs = [
    '먹고 싶은 사과',
    '먹고 싶은 바나나',
    '길고 노란 바나나 바나나',
    '저는 과일이 좋아요'
]

**CountVectorizer 이용한 실습**

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer()

In [4]:
countvect = vect.fit_transform(docs)

In [5]:
countvect.toarray()

array([[0, 0, 0, 1, 0, 1, 1, 0, 0],
       [0, 0, 0, 1, 1, 0, 1, 0, 0],
       [0, 1, 1, 0, 2, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 1, 1]])

In [8]:
sorted(vect.vocabulary_)

['과일이', '길고', '노란', '먹고', '바나나', '사과', '싶은', '저는', '좋아요']

In [12]:
import pandas as pd
countvect_df = pd.DataFrame(countvect.toarray(), columns = sorted(vect.vocabulary_))
countvect_df.index = ['문서1', '문서2', '문서3', '문서4']
countvect_df

,과일이,길고,노란,먹고,바나나,사과,싶은,저는,좋아요
문서1,0,0,0,1,0,1,1,0,0
문서2,0,0,0,1,1,0,1,0,0
문서3,0,1,1,0,2,0,0,0,0
문서4,1,0,0,0,0,0,0,1,1


In [14]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_similarity(countvect_df, countvect_df)

array([[1.        , 0.66666667, 0.        , 0.        ],
       [0.66666667, 1.        , 0.47140452, 0.        ],
       [0.        , 0.47140452, 1.        , 0.        ],
       [0.        , 0.        , 0.        , 1.        ]])

**TfidfVectorizer 이용한 실습**

In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer
vect = TfidfVectorizer()
tfvect = vect.fit_transform(docs)

In [17]:
tfidvect_df = pd.DataFrame(tfvect.toarray(), columns = sorted(vect.vocabulary_))
tfidvect_df.index = ['문서1', '문서2', '문서3', '문서4']
tfidvect_df

,과일이,길고,노란,먹고,바나나,사과,싶은,저는,좋아요
문서1,0.00000,0.00000,0.00000,0.526405,0.00000,0.667679,0.526405,0.00000,0.00000
문서2,0.00000,0.00000,0.00000,0.577350,0.57735,0.000000,0.577350,0.00000,0.00000
문서3,0.00000,0.47212,0.47212,0.000000,0.74445,0.000000,0.000000,0.00000,0.00000
문서4,0.57735,0.00000,0.00000,0.000000,0.00000,0.000000,0.000000,0.57735,0.57735


In [18]:
cosine_similarity(tfidvect_df, tfidvect_df)

array([[1.        , 0.60784064, 0.        , 0.        ],
       [0.60784064, 1.        , 0.42980824, 0.        ],
       [0.        , 0.42980824, 1.        , 0.        ],
       [0.        , 0.        , 0.        , 1.        ]])

### 3-1-2. 벡터화 실습2 - 영화 후기 데이터

In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
data = pd.read_csv('./data/movies/movies_metadata.csv', low_memory = False)
data.head(2)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0


In [3]:
data['overview'].head()

0    Led by Woody, Andy's toys live happily in his ...
1    When siblings Judy and Peter discover an encha...
2    A family wedding reignites the ancient feud be...
3    Cheated on, mistreated and stepped on, the wom...
4    Just when George Banks has recovered from his ...
Name: overview, dtype: object

In [8]:
# 전처리

data = data[data['overview'].notnull()].reset_index(drop = True)
data = data[:20000].reset_index(drop = True)

data.shape

(20000, 24)

In [9]:
tfidf = TfidfVectorizer(stop_words = 'english')
tfidf_matrix = tfidf.fit_transform(data['overview'])
tfidf_matrix.shape

(20000, 47661)

In [10]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_matrix = cosine_similarity(tfidf_matrix, tfidf_matrix)

In [11]:
# movie title과 id를 매핑할 dictionary를 생성
movie2id = {}
for i, c in enumerate(data['title']):
    movie2id[i] = c

# id와 movie title를 매핑할 dictionary를 생성
id2movie = {}
for i, c in enumerate(data['title']):
    id2movie[c] = i

In [34]:
idx = id2movie['The Dark Knight']

# 자기 자신을 제외한 영화들의 유사도 및 인덱스 추출
sim_scores = [(i, c) for i, c in enumerate(cosine_matrix[idx]) if i != idx]
sim_scores

[(0, 0.0),
 (1, 0.02163432455512368),
 (2, 0.0),
 (3, 0.0),
 (4, 0.0),
 (5, 0.0),
 (6, 0.0),
 (7, 0.0),
 (8, 0.010489269080862623),
 (9, 0.0),
 (10, 0.0),
 (11, 0.040784562550439565),
 (12, 0.0),
 (13, 0.0),
 (14, 0.0),
 (15, 0.0),
 (16, 0.0),
 (17, 0.0),
 (18, 0.007478731338598879),
 (19, 0.0),
 (20, 0.0),
 (21, 0.0),
 (22, 0.0),
 (23, 0.015680047073629723),
 (24, 0.0),
 (25, 0.0),
 (26, 0.0),
 (27, 0.0),
 (28, 0.0),
 (29, 0.018811234074033095),
 (30, 0.0),
 (31, 0.0),
 (32, 0.0069867909061646855),
 (33, 0.005134100186588265),
 (34, 0.0),
 (35, 0.0),
 (36, 0.0),
 (37, 0.0),
 (38, 0.028113783086231314),
 (39, 0.0),
 (40, 0.008733157292641168),
 (41, 0.0),
 (42, 0.025933224140665043),
 (43, 0.0),
 (44, 0.006578700163385152),
 (45, 0.0),
 (46, 0.010887452828862623),
 (47, 0.0),
 (48, 0.022076385117567307),
 (49, 0.0),
 (50, 0.0),
 (51, 0.009747046885056449),
 (52, 0.008730714379805762),
 (53, 0.0),
 (54, 0.0),
 (55, 0.013010008843501812),
 (56, 0.0),
 (57, 0.0),
 (58, 0.0),
 (59, 0.0),
 

In [35]:
# 유사도 높은 순으로 정렬
sim_scores = sorted(sim_scores, key = lambda x: x[1], reverse = True)
sim_scores[:10]

[(18144, 0.32165447841759276),
 (1314, 0.2656583251003562),
 (15444, 0.25179011096591597),
 (149, 0.2295352575035623),
 (19663, 0.20036510258964213),
 (583, 0.19010225078090667),
 (3077, 0.18726204897177956),
 (17930, 0.18543616039437819),
 (9203, 0.18301872094287325),
 (10092, 0.17146688549781672)]

In [36]:
sim_scores = [(movie2id[i], score) for i, score in sim_scores[:10]]
sim_scores

[('The Dark Knight Rises', 0.32165447841759276),
 ('Batman Returns', 0.2656583251003562),
 ('Batman: Under the Red Hood', 0.25179011096591597),
 ('Batman Forever', 0.2295352575035623),
 ('Batman: The Dark Knight Returns, Part 1', 0.20036510258964213),
 ('Batman', 0.19010225078090667),
 ('Batman: Mask of the Phantasm', 0.18726204897177956),
 ('Batman: Year One', 0.18543616039437819),
 ('Batman Beyond: Return of the Joker', 0.18301872094287325),
 ('Batman Begins', 0.17146688549781672)]

In [44]:
idx = id2movie['Harry Potter and the Philosopher\'s Stone']

# 자기 자신을 제외한 영화들의 유사도 및 인덱스 추출
sim_scores = [(i, c) for i, c in enumerate(cosine_matrix[idx]) if i != idx]
# 유사도 높은 순으로 정렬
sim_scores = sorted(sim_scores, key = lambda x: x[1], reverse = True)
sim_scores = [(movie2id[i], score) for i, score in sim_scores[:10]]
sim_scores

[('99 and 44/100% Dead', 0.19028239263013053),
 ('Harry Potter and the Goblet of Fire', 0.18099267067180552),
 ("Harry, He's Here To Help", 0.1742267754261652),
 ('Harry Potter and the Prisoner of Azkaban', 0.17379079770899322),
 ('The Dead Pool', 0.17131367680296294),
 ('Harry Potter and the Chamber of Secrets', 0.15844303564364495),
 ('Harry Potter and the Order of the Phoenix', 0.14894400146798004),
 ('Crazy Love', 0.1434263505334204),
 ("Let's Get Harry", 0.13854925019429534),
 ('Clockwise', 0.13675462524777685)]

In [42]:
idx = id2movie['Avatar']

# 자기 자신을 제외한 영화들의 유사도 및 인덱스 추출
sim_scores = [(i, c) for i, c in enumerate(cosine_matrix[idx]) if i != idx]
# 유사도 높은 순으로 정렬
sim_scores = sorted(sim_scores, key = lambda x: x[1], reverse = True)
sim_scores = [(movie2id[i], score) for i, score in sim_scores[:10]]
sim_scores

[('Project Moon Base', 0.20353879471930053),
 ('Apollo 18', 0.17707865295582864),
 ('The War of the Robots', 0.17408320270376615),
 ('The American', 0.16945012605906445),
 ('Bloodbrothers', 0.16123872958582885),
 ('Welcome to the Space Show', 0.15629834435611423),
 ('The Matrix', 0.13722999804317504),
 ('The Men', 0.12715882380454208),
 ('The Inhabited Island', 0.12568748254701914),
 ('Hellraiser: Bloodline', 0.12477861310227405)]